Імпорти

In [ ]:
import pandas as pd
import urllib.request
from datetime import datetime
from constants import *
from os import walk

Змінні

In [15]:
DIRECTORY = 'CSV_Files/'
URL = ('https://www.star.nesdis.noaa.gov/smcd/emb/vci/VH/get_TS_admin.php?country=UKR&provinceID={'
       '}&year1=1981&year2=2020&type=Mean')
INDEXES = {1: 22, 2: 24, 3: 23, 4: 25, 5: 3, 6: 4, 7: 8, 8: 19, 9: 20, 10: 21, 11: 9, 13: 10, 14: 11, 15: 12, 16: 13,
           17: 14, 18: 15, 19: 16, 21: 17, 22: 18, 24: 1, 25: 2, 26: 7, 27: 5}

Завантаження файлів

In [16]:
def download_file(province_id):
    url = URL.format(province_id)
    wp = urllib.request.urlopen(url)
    text = wp.read()
    now = datetime.now()
    date_and_time_time = now.strftime("%d-%m-%Y_%H-%M-%S")
    out = open(DIRECTORY + 'NOAA_ID' + str(province_id) + '_' + date_and_time_time + '.csv', 'wb')
    out.write(text)
    out.close()
    print('File for ' + str(province_id))
    

Створення фрейму з файлів у папці

In [17]:
def create_frame(directory):
    """
create dataframe from all files in directory
    :param directory: path to directory
    :return: dataframe with all files in directory
    """
    headers = ['Year', 'Week', 'SMN', 'SMT', 'VCI', 'TCI', 'VHI', 'empty']
    i = 1
    df_list = []
    for (dirpath, dirnames, filenames) in walk(DIRECTORY):
        for filename in filenames:
            df = pd.read_csv(directory + filename, header=1, names=headers)
            df = df.drop(df.loc[df['VHI'] == -1].index)
            df['area'] = i
            i += 1
            df_list.append(df)
    return pd.concat(df_list)

Зміна індексів

In [18]:
def change_indexes(df):
    """
change indexes of areas in dataframe    :param df:  with all files
    """
    df.replace({'area': INDEXES}, inplace=True)

Перша процедура

In [19]:
def procedure_1(df, year, province_n):
    """
display min and max values of VHI by province id and year
    :param df: dataframe with all files
    :param year: year of data
    :param province_n: province id of data
    """
    df_new = df[(df.area == province_n) & (df.VHI != -1) & (df.Year.astype(str) == str(year))]['VHI']
    print('For ' + str(year) + ' and the province ' + str(province_n) + ':')
    print('Min: ' + str(df_new.min()))
    print('Max: ' + str(df_new.max()))

Друга процедура

In [20]:
def procedure_2(df, province_n):
    """
display years with extreme drought by province id
    :param df: dataframe with all files
    :param province_n: province id of data
    """
    df_drought = df[(df["area"] == province_n) & (df.VHI <= 15) & (df.VHI != -1)]
    list_from_ds = list(df_drought.Year)
    years = []
    [years.append(item) for item in list_from_ds if item not in years]
    print('For the province ' + str(province_n) + ' extreme drought is in:')
    print('Years: ' + ' '.join(years))

Третя процедура

In [21]:
def procedure_3(df, province_n):
    """
display years with moderate drought by province id
    :param df: dataframe with all files
    :param province_n: province id of data
    """
    df_drought = df[(df["area"] == province_n) & (df.VHI <= 35) & (df.VHI > 15) & (df.VHI != -1)]
    list_from_ds = list(df_drought.Year)
    years = []
    [years.append(item) for item in list_from_ds if item not in years]
    print('For the province ' + str(province_n) + ' moderate drought is in:')
    print('Years: ' + ' '.join(years))

Завантаження всіх файлів

In [22]:
for province_n in range(1, 29):
    download_file(province_n)
print('Success')

File for 1
File for 2
File for 3
File for 4
File for 5
File for 6
File for 7
File for 8
File for 9
File for 10
File for 11
File for 12
File for 13
File for 14
File for 15
File for 16
File for 17
File for 18
File for 19
File for 20
File for 21
File for 22
File for 23
File for 24
File for 25
File for 26
File for 27
File for 28
Success


In [23]:
df = create_frame(DIRECTORY)
change_indexes(df)

In [24]:
procedure_1(df, 1990, 3)

For 1990 and the province 3:
Min: 19.5
Max: 60.62


In [25]:
procedure_2(df, 3)

For the province 3 extreme drought is in:
Years: 1986 1993


In [26]:
procedure_3(df, 3)

For the province 3 moderate drought is in:
Years: 1982 1983 1984 1985 1986 1987 1988 1989 1990 1991 1992 1993 1994 1995 1996 1997 1999 2000 2002 2003 2006 2007 2008 2010 2011 2012 2015 2017 2018 2019 2020
